In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
from PIL import Image,ImageOps
import tensorflow as tf
import cv2
from tensorflow.keras.applications import EfficientNetB4

In [4]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [380, 380])

    image=tf.keras.applications.efficientnet.preprocess_input(image)
    return image

In [5]:
dataset=pd.read_csv("/kaggle/input/labels/labels.csv")
dataset_1=pd.read_csv("/kaggle/input/aug-labels/app.csv")
dataset_1

,Unnamed: 0,name,coordinates
0,0,0.jpeg,[450.21987981 217.38683654 852.2421234 180.284...
1,1,1.jpeg,[413.38155873 214.38698589 922.49468699 142.26...
2,2,2.jpeg,[537.22301878 206.9476443 799.80094582 279.691...
3,3,3.jpeg,[386.1549854 123.67132091 775.31419938 187.219...
4,4,4.jpeg,[250.70798136 154.95323048 757.52302977 209.53...
...,...,...,...
1995,1995,1995.jpeg,[ 511.31338996 219.43527623 1001.82865173 264....
1996,1996,1996.jpeg,[405.96253025 288.89701399 704.30073223 270.87...
1997,1997,1997.jpeg,[352.10094555 259.32805278 618.39608844 222.26...
1998,1998,1998.jpeg,[377.01420068 220.82091936 835.71498715 177.78...


In [6]:
Dataset=dataset.to_numpy()
X_train=Dataset[:,0]
Y_train=Dataset[:,1]
Dataset_1=dataset_1.to_numpy()
X_train_1=Dataset_1[:,1]
Y_train_1=Dataset_1[:,2]
X_train_1

array(['0.jpeg', '1.jpeg', '2.jpeg', ..., '1997.jpeg', '1998.jpeg',
       '1999.jpeg'], dtype=object)

In [7]:
for i in range (Y_train.shape[0]):
    Y_train[i]=Y_train[i].split(",")
for i in range (Y_train.shape[0]):
    Y_train[i]=np.array(Y_train[i])
for i in range (Y_train.shape[0]):
    Y_train[i][7]=Y_train[i][7].replace(']','')
    Y_train[i][0]=Y_train[i][0].replace('[','')
Y_train=np.array(Y_train)
for i in range (Y_train.shape[0]):
    Y_train[i] = Y_train[i].astype(float)
for i in range (Y_train.shape[0]):
    Y_train[i] = Y_train[i].astype(int) 

In [8]:
Y_train_1=np.array(Y_train_1)
for i in range (Y_train_1.shape[0]):
    Y_train_1[i]=Y_train_1[i].replace(']','')
    Y_train_1[i]=Y_train_1[i].replace('[','')
for i in range (Y_train_1.shape[0]):
    Y_train_1[i]=Y_train_1[i].strip()
    Y_train_1[i]=Y_train_1[i].split(" ")
for i in range (Y_train_1.shape[0]):
    Y_train_1[i]=np.array(Y_train_1[i])
for i in range (Y_train_1.shape[0]):
    Y_train_1[i] = Y_train_1[i].astype(float)
for i in range (Y_train_1.shape[0]):
    Y_train_1[i] = Y_train_1[i].astype(int) 

In [9]:
Y=np.zeros([2000,8])
for i in range(Y.shape[0]):
    for j in range(Y.shape[1]):
        Y[i,j]=Y_train[i][j]
Y_1=np.zeros([2000,8])
for i in range(Y.shape[0]):
    for j in range(Y_train_1[i].shape[0]):
        Y_1[i,j]=Y_train_1[i][j]

In [10]:
Y_2=np.zeros([2000,8])
for i in range(2000):
    for j in range(8):
        if j%2==0:
            Y_2[i,j]=(Y[i,j]+300)*1280/1880
        else:
            Y_2[i,j]=(Y[i,j]+169)*720/1058

In [11]:
Y_3=np.zeros([2000,8])
for i in range(2000):
    for j in range(8):
        if j%2==0:
            Y_3[i,j]=(Y_1[i,j]+300)*1280/1880
        else:
            Y_3[i,j]=(Y_1[i,j]+169)*720/1058

In [31]:
batch_size = 32
img_height = 720
img_width = 1280
ds = tf.data.Dataset.from_tensor_slices('/kaggle/input/paddedimg/pad_images/'+X_train_1)
ds = ds.map(load_and_preprocess_image)
labels = tf.data.Dataset.from_tensor_slices(Y_2)
dataset_2 = tf.data.Dataset.zip((ds, labels))
dataset_2 = dataset_2.shuffle(buffer_size=32)
print(dataset_2)
num_images=2000
num_train=int(num_images*0.8)
train_data_2=dataset_2.take(num_train)
test_data_2=dataset_2.skip(num_train).take(int(num_images*0.2))

<ShuffleDataset shapes: ((380, 380, 3), (8,)), types: (tf.float32, tf.float64)>


In [32]:
batch_size = 32
img_height = 720
img_width = 1280
ds = tf.data.Dataset.from_tensor_slices('/kaggle/input/paddedimg1/pad_image(1)/'+X_train_1)
ds = ds.map(load_and_preprocess_image)
labels = tf.data.Dataset.from_tensor_slices(Y_3)
dataset_3 = tf.data.Dataset.zip((ds, labels))
dataset_3 = dataset_3.shuffle(buffer_size=32)
print(dataset_3)
num_images=2000
num_train=int(num_images*0.8)
train_data_3=dataset_3.take(num_train)
test_data_3=dataset_3.skip(num_train).take(int(num_images*0.2))

<ShuffleDataset shapes: ((380, 380, 3), (8,)), types: (tf.float32, tf.float64)>


In [33]:
batch_size = 32
img_height = 720
img_width = 1280
ds = tf.data.Dataset.from_tensor_slices('/kaggle/input/aug-images/augmented images/'+X_train_1)
ds = ds.map(load_and_preprocess_image)
labels = tf.data.Dataset.from_tensor_slices(Y_1)
dataset_1 = tf.data.Dataset.zip((ds, labels))
dataset_1 = dataset_1.shuffle(buffer_size=32)
print(dataset_1)
num_images=2000
num_train=int(num_images*0.8)
train_data_1=dataset_1.take(num_train)
test_data_1=dataset_1.skip(num_train).take(int(num_images*0.2))

<ShuffleDataset shapes: ((380, 380, 3), (8,)), types: (tf.float32, tf.float64)>


In [34]:
batch_size = 32
img_height = 720
img_width = 1280
ds = tf.data.Dataset.from_tensor_slices('/kaggle/input/images/images/'+X_train)
ds = ds.map(load_and_preprocess_image)
labels = tf.data.Dataset.from_tensor_slices(Y)
dataset = tf.data.Dataset.zip((ds, labels))
dataset = dataset.shuffle(buffer_size=32)
num_images=2000
num_train=int(num_images*0.8)
train_data=dataset.take(num_train)
test_data=dataset.skip(num_train).take(int(num_images*0.2))


In [16]:
batch_size = 32
Train_Data = train_data.concatenate(train_data_1)
Train_Data = Train_Data.concatenate(train_data_2)
Train_Data = Train_Data.concatenate(train_data_3)
Test_Data = test_data.concatenate(test_data_1)
Test_Data = Test_Data.concatenate(test_data_2)
Test_Data = Test_Data.concatenate(test_data_3)
Train_Data = Train_Data.shuffle(buffer_size=32)
Test_Data = Test_Data.shuffle(buffer_size=32)
dataset = dataset.shuffle(buffer_size=32)
Train_Data = Train_Data.batch(batch_size)
Test_Data = Test_Data.batch(batch_size)


In [26]:
# from efficientnet.keras import EfficientNetB4
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,DepthwiseConv2D,BatchNormalization,SeparableConv2D,Dropout,MaxPooling2D
from keras.models import Model

base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=(380,380,3))
x = base_model.output
x = Conv2D(2340,(3,3), activation='relu')(x)
x = Conv2D(1960,(3,3), activation='relu')(x)
x = Conv2D(1540,(3,3), activation='selu')(x)
x = Conv2D(1028,(3,3), activation='selu')(x)
x = Conv2D(720,(4,4), activation='relu')(x)
x = GlobalAveragePooling2D()(x)
x = Dense(1028, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(8, activation='linear')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam', loss='mean_squared_error',metrics='RootMeanSquaredError')
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 380, 380, 3) 0                                            
__________________________________________________________________________________________________
rescaling_3 (Rescaling)         (None, 380, 380, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
normalization_3 (Normalization) (None, 380, 380, 3)  7           rescaling_3[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 381, 381, 3)  0           normalization_3[0][0]            
____________________________________________________________________________________________

In [35]:
dataset = dataset.batch(batch_size)
dataset_1 = dataset_1.batch(batch_size)
dataset_2 = dataset_2.batch(batch_size)
dataset_3 = dataset_3.batch(batch_size)

In [29]:
model.fit(Train_Data,epochs=2,validation_data=Test_Data)

Epoch 1/2
200/200 [==============================] - 168s 837ms/step - loss: 73.6709 - root_mean_squared_error: 8.5832 - val_loss: 217.9962 - val_root_mean_squared_error: 14.7647
Epoch 2/2
200/200 [==============================] - 172s 859ms/step - loss: 93.5253 - root_mean_squared_error: 9.6708 - val_loss: 204.2181 - val_root_mean_squared_error: 14.2905


In [102]:
model.save('efficientnetB4_2.h5')

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
